In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#function that takes in a 3x3 array of coordinates for faces and returns triangles and vertices
def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])


In [4]:
#will take in and populate the soma table based on the key it gets
def axon_verts_faces(query_key):
    
    table=""
    vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
                                   & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]).fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
    else:
        vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key 
                                        & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]
                                       ).fetch("vertex_indices","triangle_indices")
        if len(vertices_soma) > 0:
            print("Soma found in Orphans")
            vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
        else:
            print("No Soma exists for " + str(query_key["segment_id"]))
            return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma).astype("int64")

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [5]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)



In [6]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)



In [7]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/pymesh_NEURITES")) == False:
    os.mkdir("pymesh_NEURITES")

In [8]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURITES"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [9]:
def meshlab_fix_manifold(key,folder="pymesh_NEURITES"):
    
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [10]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [11]:
ta3p100.Mesh.proj()

segmentation segmentation id,segment_id segment id unique within each Segmentation
1,648518346341351441
1,648518346341351458
1,648518346341351467
1,648518346341351476
1,648518346341351491
1,648518346341351495
1,648518346341351496
1,648518346341351497
1,648518346341351498
1,648518346341351500


In [12]:
# key_source = (dj.U("segment_id","segmentation") & (ta3p100.CompartmentFinal.ComponentFinal() 
#                                  & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]).proj() )

In [13]:
#shows that there are no orphans with axons
orphan_key_source = (dj.U("segment_id","segmentation") & (ta3p100.CompartmentOrphan.ComponentOrphan() 
                                 & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]).proj())

In [14]:
ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Axon-Soma'"

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,n_vertex_indices,n_triangle_indices,vertex_indices preserved indices of each vertex of this sub-compartment,triangle_indices preserved indices of each triangle of this sub-compartment


In [15]:
@schema
class NeuriteRawPostsynAxon(dj.Computed):
    definition="""
    -> ta3p100.Mesh
    ---
    vertices  :longblob #vertices for extracted axon
    triangles :longblob #faces for extracted axon
    n_edges   :int unsigned #number of edges stored
    edges     :longblob #array storing edges on each row
    n_bodies    :tinyint unsigned #the amount of segments the neurite was originally split into
    lagest_mesh_perc : float #the percentage of the entire mesh that the largest submesh makes up
     
    """
    
    
    key_source = dj.U("segment_id","segmentation") & (ta3p100.CompartmentFinal.ComponentFinal() 
                                     & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]) 

    
    def make(self, key):
        global_time = time.time()
        
        
        print(str(key["segment_id"])+ ":")
        
        #create return key
        return_key = key.copy()


        #get the vertices and triangles for the Soma
        start_time = time.time()
        vertices_whole, triangles_whole = axon_verts_faces(key)
        print(f"Step 1: extracted Soma Mesh = {time.time()-start_time}")


        #if no soma portion was found then just write regular skeleton
        if not vertices_whole.any():
            print("Error no axon")


        new_key = dict(segmentation=key["segmentation"],
                           segment_id=key["segment_id"])



        #find the bounding box
        # start_time = time.time()
        # mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
        # mesh.export(str(key["segment_id"]) + "_axon.off","off")

        #now skeletonize the mesh

        #gets all of the different parts
        mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
        count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
                                                            mesh.edges_sparse,
                                                            directed=False,
                                                            return_labels=True)

        new_key["n_bodies"] = count
        values = np.array(labels)
        searchval = 0
        ii = np.where(values == searchval)[0]
        new_key["lagest_mesh_perc"] = len(ii)/len(labels)

        print(f"Step 2a: Getting the number of splits: {time.time() - start_time}")



        start_time = time.time()
        #pass the vertices and faces to pymeshfix to become watertight
        meshfix = pymeshfix.MeshFix(vertices_whole,triangles_whole)
        meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
        print(f"Step 2b: Pymesh shrinkwrapping: {time.time() - start_time}")

        #print("Step 2: Writing Off File")
        start_time = time.time()
        #write the new mesh to off file
        path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]),meshfix.v,meshfix.f)
        print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")

        #Run the meshlabserver scripts
        start_time = time.time()
        output_mesh = meshlab_fix_manifold(key)
        print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

        print(output_mesh[:-4])

        #send to be skeletonized
        start_time = time.time()
        return_value = cm.calcification(output_mesh[:-4])
        if return_value > 0:
            raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                            ' did not finish... exited with error code: ' + str(return_value))
        #print(f"Step 5: Generating Skeleton: {time.time() - start_time}")



        #read in the skeleton files into an array
        bone_array = read_raw_skeleton(output_mesh[:-4]+"_skeleton.cgal")

        #print(bone_array)
        if len(bone_array) <= 0:
            raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
        print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")


        start_time = time.time()
        new_key["n_edges"] = len(bone_array)
        new_key["edges"] = bone_array
        
        new_key["vertices"] = vertices_whole
        new_key["triangles"] = triangles_whole
        
        self.insert1(new_key,skip_duplicates=True)
        os.system("rm "+str(path_and_filename)+"*")
        print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
        print(f"Total time: {time.time() - global_time}")
        print("\n\n")

In [16]:
#(schema.jobs & "table_name='__neurite_raw_postsyn_axon'").delete()

In [18]:
start_time = time.time()
NeuriteRawPostsynAxon.populate(reserve_jobs = True)
print(f"Total time = {time.time() - start_time}")

648518346349471565:
Soma found in Exhitatory
Step 1: extracted Soma Mesh = 0.7725458145141602
Step 2a: Getting the number of splits: 0.8530449867248535
Step 2b: Pymesh shrinkwrapping: 7.244632720947266
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.7708408832550049
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 1.924448013305664
/notebooks/14_Contacts/pymesh_NEURITES/neuron_648518346349471565_mls
Step 5: Generating and reading Skeleton: 4.161513328552246
Step 6: Inserting dictionary: 0.40302038192749023
Total time: 15.359115362167358



648518346349471910:
Soma found in Exhitatory
Step 1: extracted Soma Mesh = 0.4259974956512451
Step 2a: Getting the number of splits: 0.45524048805236816
Step 2b: Pymesh shrinkwrapping: 1.1337323188781738
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.25362348556518555
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 1.0838217735290527
/notebooks/14_Contacts/p